In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/fedbiosklearn.py'

Customize params is an optional method needed when there is type difference between fedbiomed and the scikit learn 
framework. i.e fedbiomed uses json serialization to send list objects and scikit learn SGD Regressor parameters
are of type numpy array.

In [3]:
%%writefile "$model_file"

from fedbiomed.common.fedbiosklearn import SkLearnModel
from sklearn.linear_model import SGDRegressor


class SGDRegressorTrainingPlan(SkLearnModel):
    #TODO impact of kwargs ?, is it optional ????
    def __init__(self, kwargs):
        super(SGDRegressorTrainingPlan, self).__init__(kwargs)
    
    def after_training_params(self):
        return {'coef_':  self.reg.coef_  , 'intercept_': self.reg.intercept_}
    
    def training_step(self,X,y):
        print('INITIAL MODEL PARMS COEF ',self.reg.coef_ , ' INTERCEPT ', self.reg.intercept_ )
        self.reg.partial_fit(X,y.ravel())
        print('After training coef ', self.reg.coef_ , 'Intercept ',self.reg.intercept_ )
    
    def training_data(self,batch_size = 50):
        dataset = pd.read_csv(self.dataset_path,header=None,delimiter=',')
        X = dataset.iloc[:,0:5].values
        print('X type ', type(X), ' shape ', X.shape)
        y = dataset.iloc[:,5]
        print('y type ', type(y.values), ' shape ', len(y.values))
        return (X,y.values)
    

Writing /home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/tmpkfqw_wvu/fedbiosklearn.py


In [4]:
model_args = { 'max_iter':1000, 'tol': 1e-3  }

training_args = {
    'batch_size': 50, 
    'lr': 1e-3, 
    'epochs': 5, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['sk']
rounds = 5

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='SGDRegressorTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None)

Messaging researcher_75427fbd-9918-40b8-b35c-e4c5f103f2d0 connected with result code 0
Searching for clients with data tags: ['sk'] ...
Saving file  my_model.pt  with parameters coef  [0. 0. 0. 0. 0.]  intercept_  [0.0]


In [6]:
exp.run()

Sampled clients in round  0   ['client_679668a7-7233-4257-9fd7-243c3ef561a4', 'client_40f73bcb-4b43-4735-be6f-455f0c2cf759']
researcher_75427fbd-9918-40b8-b35c-e4c5f103f2d0
researcher_75427fbd-9918-40b8-b35c-e4c5f103f2d0
Loaded coef - parms COEF (JOB)   with parameters coef  [-1.01449554  1.36640491  2.16449116  1.37427662  0.7144176 ]  intercept_  [1.00338427]
Loaded coef - parms COEF (JOB)   with parameters coef  [ 2.34228408  4.35985108  4.93259153  3.01711382 -0.24907884]  intercept_  [-0.25255903]
aggregated params  {'coef_': array([1.40436037, 3.52344701, 4.15915172, 2.55808578, 0.0201334 ]), 'intercept_': array([0.09836631])}
Saving file  /home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/researcher_params_00e3b7dd-21de-462b-b83e-4c20664b1128.pt  with parameters coef  [1.40436037 3.52344701 4.15915172 2.55808578 0.0201334 ]  intercept_  [0.09836631]
Sampled clients in round  1   ['client_679668a7-7233-4257-9fd7-243c3ef561a4', 'client_40f73bcb-4b43-47

In [7]:
n_features = 5
testing_samples = 40
rng = np.random.RandomState(1)
A = np.array([[5],
       [8],
       [9],
       [5],
       [0]])
def test_data():
    X_test = rng.randn(testing_samples, n_features).reshape([testing_samples, n_features])
    y_test = X_test.dot(A) + rng.randn(testing_samples).reshape([testing_samples,1])
    return X_test, y_test

In [8]:
from sklearn.linear_model import SGDRegressor

In [9]:

X_test, y_test = test_data()

In [10]:
X_test.shape

(40, 5)

In [11]:
y_test.shape

(40, 1)

In [11]:
( exp._aggregated_params[0]['params'] , exp._aggregated_params[0]['weights'] )

({'coef_': array([-0.02739925,  0.0234013 , -0.01150177, -0.00410545,  0.41344659]),
  'intercept_': array([0.04178598])},
 [0.27941176470588236, 0.7205882352941176])

In [13]:
exp._aggregated_params[1]['params']

{'coef_': array([-0.03240349,  0.03181308, -0.01020819,  0.01679504,  0.56607199]),
 'intercept_': array([0.07960475])}

In [14]:
exp._aggregated_params[2]['params']

{'coef_': array([-0.03286285,  0.03248214, -0.00932428,  0.01935818,  0.65534029]),
 'intercept_': array([0.09097783])}

In [15]:
exp._aggregated_params[3]['params']

{'coef_': array([-3.52147691e-02,  2.57360185e-02, -6.31080165e-04,  4.84028682e-03,
         7.41359561e-01]),
 'intercept_': array([0.07079504])}

In [16]:
exp._aggregated_params[4]['params']

{'coef_': array([-0.03459867,  0.02048709,  0.00353996, -0.00253229,  0.80389615]),
 'intercept_': array([0.05667927])}

In [12]:
import matplotlib.pyplot as plt

In [10]:
testing_error = []

for i in range(rounds):
    fed_model = SGDRegressor(max_iter=1000, tol=1e-3)
    print(id(fed_model))
    fed_model.coef_ = exp._aggregated_params[i]['params']['coef_'].copy()
    fed_model.intercept_ = exp._aggregated_params[i]['params']['intercept_'].copy()
    
    mse = np.mean((fed_model.predict(X_test).ravel() - y_test.ravel())**2)
    print('MSE ', mse)
    testing_error.append(mse)

139757150180064
MSE  54.240592733561904
139757150620352
MSE  36.78866327899565
139757150621168
MSE  18.65328969207578
139757150618912
MSE  10.613543402156429
139757150180112
MSE  8.580333296143738


In [12]:
testing_error

[54.240592733561904,
 36.78866327899565,
 18.65328969207578,
 10.613543402156429,
 8.580333296143738]

In [11]:
plt.plot(testing_error, label = 'federated test error')
plt.hlines(np.mean((X_test.dot(A).ravel() - y_test.ravel())**2), 0, N_rounds-1, color = 'red', alpha = 0.7, label = 'ground truth')
plt.xlabel('Round')
plt.legend()

NameError: name 'plt' is not defined

In [10]:
exp._aggregated_params[0]['params']['coef_']

array([-0.08835477, -0.02177115,  0.09723794,  0.03879841,  0.00388391])

In [11]:
exp._aggregated_params[1]['params']['coef_']

array([-0.0857399 , -0.02418705,  0.09421684,  0.04130261,  0.00110738])

In [12]:
exp._aggregated_params[2]['params']['coef_']

array([-0.08628797, -0.02662039,  0.09914285,  0.04398299,  0.00211929])

In [13]:
exp._aggregated_params[3]['params']['coef_']

array([-0.0876811 , -0.02231482,  0.09693187,  0.04262923,  0.00136258])

In [ ]:
exp._aggregated_params[1]['params']['coef_']

In [ ]:
exp._aggregated_params[1]['params']['coef_']

In [8]:
exp._aggregated_params[1]

{'params': {'coef_': array([-0.09106583, -0.02673528,  0.09718337,  0.04359848, -0.00150982]),
  'intercept_': array([0.02567093])},
 'params_path': '/home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/researcher_params_b98dd684-2020-4337-a061-0e1f7989c33d.pt'}

In [9]:
exp._aggregated_params[2]

{'params': {'coef_': array([-0.08315385, -0.02271357,  0.09840193,  0.04169297,  0.00091099]),
  'intercept_': array([0.02895132])},
 'params_path': '/home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/researcher_params_e4a565be-dfe9-42a3-8d45-96915f403bd7.pt'}

In [ ]:
%%writefile "$model_file"

from fedbiomed.common.fedbiosklearn import SkLearnModel
from sklearn.linear_model import SGDRegressor


class SGDRegressorTrainingPlan(SkLearnModel):
    #TODO impact of kwargs ?, is it optional ????
    def __init__(self, kwargs):
        super(SGDRegressorTrainingPlan, self).__init__()
        self.reg = SGDRegressor(max_iter=kwargs['max_iter'], tol=kwargs['tol'])  
        if(kwargs.get('coef_') is not None and kwargs.get('intercept_') is not None):
            if isinstance(kwargs['coef_'],list) and isinstance(kwargs['intercept_'],list):
                self.reg.coef_ = np.array(kwargs['coef_'])
                self.reg.intercept_ = np.array(kwargs['intercept_'])
    
    
    def customize_params(self,kwargs):
        
    
    def after_training_params(self):
        return {'coef_':  self.reg.coef_  , 'intercept_': self.reg.intercept_}
    
    def training_step(self,X,y):
        self.reg.partial_fit(X,y)
        print('After training coef ', self.reg.coef_ , 'Intercept ',self.reg.intercept_ )
    
    def training_data(self,batch_size = 48):
        n_samples, n_features = 100, 5
        testing_samples = 40

        ### Creating a random dataset
        rng = np.random.RandomState(1)

        y = []
        X = []

        ### Creating a random linear transformation
        A = rng.randint(0,10,size = n_features).reshape([n_features,1])

        ### For every center we create random X and y dataset with same generative rule (the matrix A)

        X = rng.randn(n_samples, n_features)
        y = rng.randn(n_samples)
        print('X shape' , X.shape , 'y shape', y.shape)
        return (X,y)